In [ ]:
# 1. Carga y unión de datos
import pandas as pd
import os

# Definir ruta base
base_path = r"D:\HP\Downloads\Ganancias de los eSports"

# Cargar archivos CSV
df_players = pd.read_csv(os.path.join(base_path, "highest_earning_players.csv"))
df_teams = pd.read_csv(os.path.join(base_path, "highest_earning_teams.csv"))
df_country_codes = pd.read_csv(os.path.join(base_path, "country-and-continent-codes-list.csv"))

# Explorar brevemente
print(df_players.head())
print(df_teams.head())
print(df_country_codes.head())


In [ ]:
# 2. Análisis Exploratorio de Datos (EDA)

# Información general y tipos de datos
print(df_players.info())

# Estadísticas descriptivas de variables numéricas
print(df_players.describe())

# Revisión de valores faltantes
print(df_players.isnull().sum())

import matplotlib.pyplot as plt
import seaborn as sns

# Distribución de la variable objetivo
sns.histplot(df_players['TotalUSDPrize'], bins=50, kde=True)
plt.title("Distribución de Ganancias Totales")
plt.xlabel("Ganancias Totales (USD)")
plt.ylabel("Frecuencia")
plt.show()

# Análisis de variables categóricas
categorical_cols = ['Game', 'Genre', 'CountryCode']

for col in categorical_cols:
    print(f"Valores únicos en {col}: {df_players[col].nunique()}")
    plt.figure(figsize=(8,6))
    sns.countplot(y=df_players[col], order=df_players[col].value_counts().index)
    plt.title(f"Distribución de {col}")
    plt.xlabel("Cantidad de jugadores")
    plt.ylabel(col)
    plt.show()


In [ ]:
#3. Feature Engineering y Selección

features = ['Game', 'Genre', 'CountryCode']
target = 'TotalUSDPrize'

df_encoded = pd.get_dummies(df_players[features], drop_first=True)

# Juntar con la variable objetivo
df_model = pd.concat([df_encoded, df_players[target]], axis=1)

print("DataFrame listo para el modelo:")
print(df_model.head())

In [ ]:
# 4. División de datos
from sklearn.model_selection import train_test_split

# Separar X (features) e y (target)
X = df_model.drop(columns=[target])
y = df_model[target]

# Dividir datos: 80% train, 20% test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {X_train.shape[0]} registros")
print(f"Tamaño del conjunto de prueba: {X_test.shape[0]} registros")


In [ ]:
# 5.1 Entrenar Decision Tree Regressor (Modelo Base)
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# Inicializar el modelo con un límite de profundidad para evitar que se ajuste demasiado a los datos de entrenamiento
dt_model = DecisionTreeRegressor(random_state=42, max_depth=10)

# Entrenar el modelo con los datos de entrenamiento
dt_model.fit(X_train, y_train)

# Realizar predicciones en los datos de prueba
y_pred_test = dt_model.predict(X_test)

# Evaluar el rendimiento del modelo en el conjunto de prueba
print("--- Resultados del Decision Tree ---")
# RMSE indica el error promedio de las predicciones en USD. Un valor más bajo es mejor.
print(f"Error Cuadrático Medio (RMSE): ${np.sqrt(mean_squared_error(y_test, y_pred_test)):.2f}")

# R2 (Coeficiente de Determinación) indica qué porcentaje de la varianza de las ganancias es explicado por el modelo. Un valor más cercano a 1 es mejor.
print(f"Coeficiente de Determinación (R2): {r2_score(y_test, y_pred_test):.4f}")

In [ ]:
# 5.2 Entrenar Random Forest Regressor (Modelo Avanzado)
from sklearn.ensemble import RandomForestRegressor

# Inicializar el modelo.
rf_model = RandomForestRegressor(random_state=42, n_estimators=100, max_depth=15, n_jobs=-1)

# Entrenar el modelo
rf_model.fit(X_train, y_train)

# Realizar predicciones en los datos de prueba
y_pred_test_rf = rf_model.predict(X_test)

# Evaluar el rendimiento del modelo
print("\n--- Resultados del Random Forest ---")
print(f"Error Cuadrático Medio (RMSE): ${np.sqrt(mean_squared_error(y_test, y_pred_test_rf)):.2f}")
print(f"Coeficiente de Determinación (R2): {r2_score(y_test, y_pred_test_rf):.4f}")


In [ ]:
# 6. Visualización de resultados
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Gráfico 1: Comparar predicciones vs valores reales para Random Forest
plt.figure(figsize=(10, 8))

sns.scatterplot(x=y_test, y=y_pred_test_rf, alpha=0.6)
# Se añade una línea roja. Si los puntos se acercan a esta línea, la predicción es buena.
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.title("Random Forest: Ganancias Reales vs. Predichas", fontsize=15)
plt.xlabel("Ganancias Reales (USD)", fontsize=12)
plt.ylabel("Ganancias Predichas (USD)", fontsize=12)
plt.show()

# Gráfico 2: Importancia de características del Random Forest
importances = rf_model.feature_importances_
features_names = X.columns
feat_imp = pd.Series(importances, index=features_names).sort_values(ascending=False)

# Seleccionamos solo las 20 características más importantes para visualizar
top_features = feat_imp.head(20)

plt.figure(figsize=(12, 8))
sns.barplot(x=top_features, y=top_features.index)
plt.title("Top 20 Características más Importantes - Random Forest", fontsize=15)
plt.xlabel("Importancia Relativa", fontsize=12)
plt.ylabel("Característica", fontsize=12)
plt.show()

In [ ]:
# 7. Visualización de Resultados sin Outliers
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Para facilitar el filtrado, creamos un DataFrame temporal con los valores reales y predichos.
df_plot = pd.DataFrame({
    'Ganancias Reales': y_test,
    'Ganancias Predichas': y_pred_test_rf
})

# Calculamos el umbral para considerar un outlier (el 5% de los jugadores con mayores ganancias)
# Usamos el cuantil 0.95, que nos da el valor por debajo del cual se encuentra el 95% de los datos.
outlier_threshold = df_plot['Ganancias Reales'].quantile(0.95)
print(f"Se filtrarán para la visualización las ganancias reales superiores a: ${outlier_threshold:,.2f} USD")

# Filtramos el DataFrame para excluir los outliers
df_filtered_plot = df_plot[df_plot['Ganancias Reales'] < outlier_threshold]


# Generamos el nuevo gráfico con los datos filtrados
plt.figure(figsize=(10, 8))
sns.scatterplot(
    x='Ganancias Reales',
    y='Ganancias Predichas',
    data=df_filtered_plot,
    alpha=0.6
)

# La línea de predicción perfecta se ajusta ahora a los nuevos límites de los datos filtrados
plt.plot(
    [df_filtered_plot['Ganancias Reales'].min(), df_filtered_plot['Ganancias Reales'].max()],
    [df_filtered_plot['Ganancias Reales'].min(), df_filtered_plot['Ganancias Reales'].max()],
    'r--', lw=2
)

plt.title("Ganancias Reales vs. Predichas (Sin el 5% de Outliers Superiores)", fontsize=15)
plt.xlabel("Ganancias Reales (USD)", fontsize=12)
plt.ylabel("Ganancias Predichas (USD)", fontsize=12)
plt.show()

In [ ]:
!conda env export > environment.yml